In [1]:
import json

file_path = '/home/daniil/Документы/like.json'


with open(file_path, 'r') as file:
    like_dataset = json.load(file)

sample_key = next(iter(like_dataset.keys()))
like_dataset[sample_key]


{'bboxes': [[0.34835416, 0.86326408, 0.25937015, 0.12973758],
  [0.5818777, 0.44759481, 0.16477829, 0.16282391]],
 'labels': ['no_gesture', 'like'],
 'landmarks': [[[0.35544432328385095, 0.9178467912811518],
   [0.3895571133807179, 0.8888129819044327],
   [0.42687296926606566, 0.8765239573841337],
   [0.46768826637141475, 0.881365368753095],
   [0.49627313599836925, 0.8929987321938916],
   [0.45034106384097305, 0.9353003075967865],
   [0.5149161960109939, 0.938208266694159],
   [0.5508345467304484, 0.9381142225764555],
   [0.5785308318092804, 0.9404011979190876],
   [0.4469011545352173, 0.9599136798577973],
   [0.5142024952392664, 0.9645932494530195],
   [0.5538340563106309, 0.9668814567025878],
   [0.5833119247159521, 0.9699959142001356],
   [0.4390902744418257, 0.9747603998248608],
   [0.49704085089653727, 0.9841366488338632],
   [0.5334690050798111, 0.9890552768139765],
   [0.560818492178926, 0.992685381468313],
   [0.4304805992564116, 0.9821572823641742],
   [0.4742554972876004, 0.

In [2]:

landmark_lengths = []

for key, value in like_dataset.items():
    if value['labels'] and value['landmarks']:
        for landmarks in value['landmarks']:
            landmark_lengths.append(len(landmarks))


unique_lengths = set(landmark_lengths)
unique_lengths


{0, 21}

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np


cleaned_features = []
cleaned_labels = []

for key, value in like_dataset.items():
    if value['labels'] and value['landmarks']:
        index = 0 if value['leading_hand'] == 'left' else 1
        if index < len(value['landmarks']) and len(value['landmarks'][index]) == 21:
            cleaned_features.append(np.array(value['landmarks'][index]).flatten())
            cleaned_labels.append(1 if 'like' in value['labels'][index] else 0)

cleaned_features = np.array(cleaned_features)
cleaned_labels = np.array(cleaned_labels)

X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    cleaned_features, cleaned_labels, test_size=0.25, random_state=42)


model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train_cleaned, y_train_cleaned)


accuracy_rf = model_rf.score(X_test_cleaned, y_test_cleaned)
accuracy_rf



0.9473684210526315

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.ensemble import RandomForestClassifier


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Игнорирование пустого кадра камеры.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

       
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark]).flatten().reshape(1, -1)
                gesture_prediction = model_rf.predict(landmarks)
                gesture_label = '+' if gesture_prediction == 1 else '-'

               
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS)
                cv2.putText(image, gesture_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


2024-04-18 16:33:20.085810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 16:33:20.833334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0000 00:00:1713447202.136738    5030 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713447202.140020    5135 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: REMBRANDT (rembrandt, LLVM 15.0.7, DRM 3.54, 6.5.0-27-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
